# <mark>**MAGIC SAUCE FOR ProjectPhaseGrossMarginFACT**</mark>

<mark></mark>

In [35]:
Declare @ProjectID Varchar(50)
Set @ProjectID = '22221929';
Declare @Company Varchar(50)
Set @Company = 'Solar';

--Get Open Job Demands for Project for each job Assembly and Material sequence for each Order Line
With OpenJobDemandsDetail AS
(Select 
    Company, 
    ProjectID,
    OrderNum,
    OrderLine,
    (Case
         When (EstUnitCost*(RequiredQty-IssuedQty)) >= 0 Then  (EstUnitCost*(RequiredQty-IssuedQty))
         Else 0
    ENd) as OpenJobDemands,
    (EstUnitCost*RequiredQty) as EstimatedJobDemands
FROM [ERPDB].[etl].[vwJobMtlChange]
WHERE 1=1
/*******************************************************************
Had to put these In here because of an error getting returned on 
converting Varchar to INT in this part of the query.  How to Solve?
*******************************************************************/
    and ProjectID = @ProjectID
    and Company = @Company
),



--Summarize the Open Job Demands by orderline for comparison vs the actual costs on each order line
OpenJobDemands AS
(Select 
    Company,
    ProjectID,
    OrderNum,
    OrderLine,
    SUM(OpenJobDemands) as OpenJobDemands,
    SUM(EstimatedJobDemands) as EstimatedJobDemands
From OpenJobDemandsDetail
Group by Company, ProjectID, OrderNum, OrderLine
),

ProjectPhaseOrderLines as
(SELECT DISTINCT
    pp.Company,
    pp.ProjectID,
    Case
        When CHARINDEX('.',pp.PhaseID) = 0 then LEN(pp.PhaseID)
        ELSE LEFT(pp.PhaseID,CHARINDEX('.',pp.PhaseID) - 1) 
    END as ParentPhaseID,
    pp.OrderNum,
    pp.OrderLine,
    SUM(ojd.OpenJobDemands) as OpenJobDemands,
    SUM(ojd.EstimatedJobDemands) as EstimatedJobDemands
FROM ERPDB.ERP.ProjPhase pp
LEFT JOIN OpenJobDemands ojd
    On pp.Company = ojd.Company and pp.OrderNum = ojd.OrderNum and pp.OrderLine = ojd.OrderLine
WHERE 1=1
    and pp.ProjectID = @ProjectID
    and pp.Company = @Company
    and pp.OrderNum <> 0
GROUP BY pp.Company, pp.ProjectID, pp.PhaseID, pp.OrderNum, pp.OrderLine
),

ProjectPhaseDemands as
(Select 
    ppol.Company,
    ppol.ProjectID,
    ppol.ParentPhaseID,
    SUM(ppol.OpenJobDemands) as OpenJobDemands,
    SUM(ppol.EstimatedJobDemands) as EstimatedJobDemands
FROM ProjectPhaseOrderLines  ppol
Group by Company, ProjectID, ParentPhaseID
),

ProjectPhaseRollup AS
(SELECT    
        Company,
        ProjectID,
        PhaseID,
        [Description],
        OrderNum,
        OrderLine,
        SUM(BudTotLbrCost + BudTotBurCost + BudTotSubCost + BudTotMtlCost + BudTotMtlBurCost + BudTotODCCost) as BudgetedCosts,
        SUM(TotQuotODCCost + TotQuotLbrCost + TotQuotBurCost + TotQuotSubContCost + TotQuotMtlCost+ TotQuotMtlBurCost) as QuotedCosts,
        SUM([TotActLbrCost]+[TotActBurCost]+[TotActSubContCost]+[TotActMtlCost]+[TotActMtlBurCost]+[TotActODCCost]) as ActualCost,
        SUM(TotWBSPhaseRev) as SellPrice
FROM ERPDB.ERP.ProjPhase
    -- Where clause for Level only includes project phases at the top level which are already Summarized from lower levels
WHERE Level = 1
GROUP BY Company, ProjectID, PhaseID, [Description], OrderNum, OrderLine
)

SELECT 
       pp.Company,
       pp.ProjectID,
       pp.PhaseID,
       pp.[Description],
       --pp.OrderNum,
       --pp.OrderLine,
       SUM(pp.SellPrice) as TotalSellPrice,
       SUM(pp.QuotedCosts) as TotalQuotedCosts,
       SUM(pp.ActualCost) as TotalActualCosts,
       SUM(pp.BudgetedCosts) as TotalBudgetedCosts,
       SUM(COALESCE(ppd.OpenJobDemands,0)) as OpenJobDemands,
       SUM(COALESCE(ppd.OpenJobDemands,0) + COALESCE(pp.ActualCost,0)) as ActualCostsandOpenJobDemands,
       SUM(COALESCE(ppd.EstimatedJobDemands,0)) as EstimatedCosts
FROM ProjectPhaseRollup pp
LEFT JOIN ProjectPhaseDemands ppd
    ON pp.ProjectID = ppd.ProjectID and pp.PhaseID = ppd.ParentPhaseID and pp.Company = ppd.Company
WHERE 1=1
    and pp.ProjectID = @ProjectID
    and pp.Company = @Company
GROUP BY pp.Company, pp.ProjectID, pp.PhaseID, pp.[Description]



Warning: Null value is eliminated by an aggregate or other SET operation.

(5 rows affected)

Total execution time: 00:00:08.191

Company,ProjectID,PhaseID,Description,TotalSellPrice,TotalQuotedCosts,TotalActualCosts,TotalBudgetedCosts,OpenJobDemands,ActualCostsandOpenJobDemands,EstimatedCosts
SOLAR,22221929,,Root Phase,0.000,0.000,0.000,0.000,0.0000000000,0.000,0.00000000000
SOLAR,22221929,1,Design,1250.000,400.000,0.000,400.000,0.0000000000,0.000,0.00000000000
SOLAR,22221929,2,Material,530213.400,378708.450,279321.180,318959.050,766.2980000000,280087.478,64221.69528000000
SOLAR,22221929,3,Freight,74266.000,57345.820,40473.210,57345.820,0.0000000000,40473.210,0.00000000000
SOLAR,22221929,4,Install,116100.000,78762.030,99615.270,98762.030,0.0000000000,99615.270,0.00000000000


**Joining the ProjectPhase Estimated Costs on the OrderLine Detail**

In [1]:
Declare @ProjectID Varchar(50)
Set @ProjectID = '22221929';
Declare @Company Varchar(50)
Set @Company = 'Solar';

With ProjectPhaseEst
AS (SELECT Company,
           ProjectID,
           OrderNum,
           OrderLine,
           Sum(BudTotLbrCost) as BudgetedLaborCosts,
           Sum(BudTotBurCost) as BudgetedBurdenCosts,
           Sum(BudTotSubCost) as BudgetedSubCosts,
           Sum(BudTotMtlCost) as BudgetedMaterialCosts,
           Sum(BudTotMtlBurCost) as BudgetedMaterialBurdenCosts,
           Sum(BudTotODCCost) as BudgetedOtherDirectCosts,
           Sum(BudTotLbrCost + BudTotBurCost + BudTotSubCost + BudTotMtlCost + BudTotMtlBurCost + BudTotODCCost) as EstimatedCosts
    From ERPDB.ERP.ProjPhase
    -- Where clause for Level only includes project phases at the top level which are already summarized from lower levels
    Where Level = 1
    Group By Company,
             ProjectID,
             OrderNum,
             OrderLine
   )

Select od.*,
       pp.EstimatedCosts
FROM [ERPDB].[etl].[vwOrderdetail] as od
    LEFT Join ProjectPhaseEst as pp
        ON od.OrderNum = pp.OrderNum
           and od.OrderLine = pp.Orderline
Where od.ProjectID = @ProjectID
      and od.company = @Company

(5 rows affected)

Total execution time: 00:00:00.118

VoidLine,OpenLine,Company,OrderNum,OrderLine,OrderNumLine,LineType,PartNum,LineDesc,Reference,IUM,RevisionNum,POLine,Commissionable,DiscountPercent,UnitPrice,DocUnitPrice,OrderQty,Discount,DocDiscount,RequestDate,ProdCode,XPartNum,XRevisionNum,PricePerCode,OrderComment,ShipComment,InvoiceComment,PickListComment,TaxCatID,AdvanceBillBal,DocAdvanceBillBal,QuoteNum,QuoteLine,TMBilling,OrigWhyNoTax,NeedByDate,CustNum,Rework,RMANum,RMALine,ProjectID,ContractNum,ContractCode,BasePartNum,Warranty,WarrantyCode,MaterialDuration,LaborDuration,MiscDuration,MaterialMod,LaborMod,WarrantyComment,Onsite,MatCovered,LabCovered,MiscCovered,SalesUM,SellingFactor,SellingQuantity,SalesCatID,ShipLineComplete,CumeQty,CumeDate,MktgCampaignID,MktgEvntSeq,LockQty,Linked,ICPONum,ICPOLine,ExtCompany,LastConfigDate,LastConfigTime,LastConfigUserID,ConfigUnitPrice,ConfigBaseUnitPrice,PriceListCode,BreakListCode,PricingQty,LockPrice,ListPrice,DocListPrice,OrdBasedPrice,DocOrdBasedPrice,PriceGroupCode,OverridePriceList,BaseRevisionNum,PricingValue,DisplaySeq,KitParentLine,KitAllowUpdate,KitShipComplete,KitBackFlush,KitPrintCompsPS,KitPrintCompsInv,KitPricing,KitQtyPer,SellingFactorDirection,RepRate1,RepRate2,RepRate3,RepRate4,RepRate5,RepSplit1,RepSplit2,RepSplit3,RepSplit4,RepSplit5,DemandContractLine,CreateNewJob,DoNotShipBeforeDate,GetDtls,DoNotShipAfterDate,SchedJob,RelJob,EnableCreateNewJob,EnableGetDtls,EnableSchedJob,EnableRelJob,CounterSaleWarehouse,CounterSaleBinNum,CounterSaleLotNum,CounterSaleDimCode,DemandDtlRejected,KitFlag,KitsLoaded,DemandContractNum,DemandHeadSeq,DemandDtlSeq,ChangedBy,ChangeDate,ChangeTime,ChangeDateTime,ReverseCharge,TotalReleases,Rpt1UnitPrice,Rpt2UnitPrice,Rpt3UnitPrice,Rpt1Discount,Rpt2Discount,Rpt3Discount,Rpt1AdvanceBillBal,Rpt2AdvanceBillBal,Rpt3AdvanceBillBal,Rpt1ListPrice,Rpt2ListPrice,Rpt3ListPrice,Rpt1OrdBasedPrice,Rpt2OrdBasedPrice,Rpt3OrdBasedPrice,ExtPriceDtl,DocExtPriceDtl,Rpt1ExtPriceDtl,Rpt2ExtPriceDtl,Rpt3ExtPriceDtl,LineStatus,InUnitPrice,DocInUnitPrice,InDiscount,DocInDiscount,InListPrice,DocInListPrice,InOrdBasedPrice,DocInOrdBasedPrice,Rpt1InUnitPrice,Rpt2InUnitPrice,Rpt3InUnitPrice,Rpt1InDiscount,Rpt2InDiscount,Rpt3InDiscount,Rpt1InListPrice,Rpt2InListPrice,Rpt3InListPrice,Rpt1InOrdBasedPrice,Rpt2InOrdBasedPrice,Rpt3InOrdBasedPrice,InExtPriceDtl,DocInExtPriceDtl,Rpt1InExtPriceDtl,Rpt2InExtPriceDtl,Rpt3InExtPriceDtl,OldOurOpenQty,OldSellingOpenQty,OldOpenValue,OldProdCode,PrevSellQty,PrevPartNum,PrevXPartNum,KitCompOrigSeq,KitCompOrigPart,SmartStringProcessed,SmartString,RenewalNbr,DiscBreakListCode,DiscListPrice,LockDisc,OverrideDiscPriceList,GroupSeq,ECCOrderNum,ECCOrderLine,DupOnJobCrt,UndersPct,Overs,Unders,OversUnitPrice,PlanUserID,PlanGUID,MOMsourceType,MOMsourceEst,DefaultOversPricing,ECCPlant,ECCQuoteNum,ECCQuoteLine,SysRevID,SysRowID,MfgJobType,ProFormaInvComment,CreateJob,ContractID,LinkToContract,DocInAdvanceBillBal,InAdvanceBillBal,Rpt1InAdvanceBillBal,Rpt2InAdvanceBillBal,Rpt3InAdvanceBillBal,PCLinkRemoved,CommodityCode,MSRP,DocMSRP,Rpt1MSRP,Rpt2MSRP,Rpt3MSRP,EndCustomerPrice,DocEndCustomerPrice,Rpt1EndCustomerPrice,Rpt2EndCustomerPrice,Rpt3EndCustomerPrice,PromotionalPrice,DocPromotionalPrice,Rpt1PromotionalPrice,Rpt2PromotionalPrice,Rpt3PromotionalPrice,OrderLineStatusCode,AttributeSetID,ForeignSysRowID,UD_SysRevID,DoNotBook_c,EstUnitCost_c,OriginalEstimate_c,EstimatedCosts
0,1,SOLAR,4000149,1,4000149-1,PART,DESIGN,DESIGN,,EA,C,,1,0.00,8750.00000,8750.00000,1.00000000,0.000,0.000,2022-07-15,02DES,,,E,,,,,,0.00000,0.00000,10144,1,0,,2022-07-31,1180,0,0,0,2128153,0,,,1,,0,0,0,,,,0,0,0,0,EA,1.00000000,1.00000000,AGIL,0,0.00000000,NULL,DEFAULT,1,0,0,0,0,,NULL,0,,0.00000,0.00000,,,0.00000000,0,8750.00000,8750.00000,8750.00000,8750.00000,,0,,0.000,1.000,1,0,0,0,0,0,,0.00000000,D,0.00,0.00,0.00,0.00,0.00,100,0,0,0,0,0,1,NULL,0,NULL,0,0,1,1,1,1,,,,,0,,0,0,0,0,LS,2022-07-04,54382,2022-07-04 15:06:22.000,0,1,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000

**Sell Price Query Example**

In [7]:
Declare @ProjectID Varchar(50)
Set @ProjectID = '22221929';
Declare @Company Varchar(50)
Set @Company = 'Solar';

With OrderDetail
as (Select od.Company,
           od.OrderNum,
           od.ProjectID,
           od.OrderLine,
           od.UnitPrice,
           od.OrderQty,
           (od.UnitPrice * od.OrderQty) as OrderLineSellPrice
    FROM [ERPDB].[etl].[vwOrderdetail] as od
   )

Select Company,
       ProjectID,
       Sum(OrderLineSellPrice) as SellPrice
From OrderDetail
Where ProjectID = @ProjectID
      and company = @Company
Group By Company,
         ProjectID


(1 row affected)

Total execution time: 00:00:00.057

Company,ProjectID,SellPrice
SOLAR,2128153,22422207.00000000000


**Quoted Cost Query Example**

In [9]:
Declare @ProjectID Varchar(50)
Set @ProjectID = '22221929';
Declare @Company Varchar(50)
Set @Company = 'Solar';

With ProjectPhaseEst
AS (SELECT Company,
           ProjectID,
           OrderNum,
           OrderLine,
           Sum(BudTotLbrCost + BudTotBurCost + BudTotSubCost + BudTotMtlCost + BudTotMtlBurCost + BudTotODCCost) as EstimatedCosts,
           Sum(TotQuotODCCost) as QuotedOtherDirectCosts,
           Sum(TotQuotODCCost + TotQuotLbrCost + TotQuotBurCost + TotQuotSubContCost + TotQuotMtlCost
               + TotQuotMtlBurCost
              ) as QuotedCosts
    From ERPDB.ERP.ProjPhase
    -- Where clause for Level only includes project phases at the top level which are already summarized from lower levels
    Where Level = 1
    Group By Company,
             ProjectID,
             OrderNum,
             OrderLine
   )

Select od.Company,
       od.ProjectID,
       sum(pp.QuotedOtherDirectCosts) as TotalQuotedOtherDirectCosts,
       sum(pp.QuotedCosts) as TotalQuotedCosts
FROM [ERPDB].[etl].[vwOrderdetail] as od
    LEFT Join ProjectPhaseEst as pp
        ON od.OrderNum = pp.OrderNum
           and od.OrderLine = pp.Orderline
Where od.ProjectID = @ProjectID
      and od.Company = @Company
Group by od.ProjectID,
         od.Company

Warning: Null value is eliminated by an aggregate or other SET operation.

(1 row affected)

Total execution time: 00:00:00.088

Company,ProjectID,TotalQuotedOtherDirectCosts,TotalQuotedCosts
SOLAR,22221929,515216.300,515216.300


**Current Budget Query Example**

In [8]:
Declare @ProjectID Varchar(50)
Set @ProjectID = '22221929';
Declare @Company Varchar(50)
Set @Company = 'Solar';

With ProjectPhaseEst
AS (SELECT Company,
           ProjectID,
           OrderNum,
           OrderLine,
           Sum(BudTotLbrCost) as BudgetedLaborCosts,
           Sum(BudTotBurCost) as BudgetedBurdenCosts,
           Sum(BudTotSubCost) as BudgetedSubCosts,
           Sum(BudTotMtlCost) as BudgetedMaterialCosts,
           Sum(BudTotMtlBurCost) as BudgetedMaterialBurdenCosts,
           Sum(BudTotODCCost) as BudgetedOtherDirectCosts,
           Sum(BudTotLbrCost + BudTotBurCost + BudTotSubCost + BudTotMtlCost + BudTotMtlBurCost + BudTotODCCost) as BudgetedCosts
    From ERPDB.ERP.ProjPhase
    -- Where clause for Level only includes project phases at the top level which are already summarized from lower levels
    Where Level = 1
    Group By Company,
             ProjectID,
             OrderNum,
             OrderLine
   )

Select od.Company,
       od.ProjectID,
       sum(pp.BudgetedCosts) as TotalBudgetedCosts
FROM [ERPDB].[etl].[vwOrderdetail] as od
    LEFT Join ProjectPhaseEst as pp
        ON od.OrderNum = pp.OrderNum
           and od.OrderLine = pp.Orderline
Where od.ProjectID = @ProjectID
      and od.Company = @Company
Group by od.ProjectID,
         od.Company

Warning: Null value is eliminated by an aggregate or other SET operation.

(1 row affected)

Total execution time: 00:00:00.158

Company,ProjectID,TotalBudgetedCosts
SOLAR,22221929,466582.300


**Actual Cost Query Example** for High River Project

In [1]:
Declare @ProjectID Varchar(50)
Set @ProjectID = '22221929';
Declare @Company Varchar(50)
Set @Company = 'Solar';

with PartSummary
as (Select Company,
       ProjectID,
       OrderNum,
       Ordernumline,
       Sum(ExtCost) as OrderLinePartCosts
FROM [ERPDB].[etl].[vwPartTran]
Group by Company,
         ProjectID,
         OrderNum,
         OrderNumLine
),

LaborSummary
as (Select Company,
       ProjectID,
       OrderNum,
       Ordernumline,
       Sum(LaborHrs*LaborRate) as LaborCost,
       Sum(BurdenHrs*BurdenRate) as BurdenCost,
       Sum((LaborHrs*LaborRate)+(BurdenHrs*BurdenRate)) as OrderLineLaborCosts
FROM [ERPDB].[etl].[vwLaborDtl]
Group by Company,
         ProjectID,
         OrderNum,
         OrderNumLine
)

Select od.Company,
    od.ProjectID,
 --   od.OrderNumline,
    Sum(ps.OrderLinePartCosts) as TotalPartCosts,
    Sum(ls.OrderLineLaborCosts) as TotalLaborCosts,
    Sum(COALESCE(ps.OrderLinePartCosts,0))+Sum(COALESCE(ls.OrderLineLaborCosts,0)) as TotalCosts
From [ERPDB].[etl].[vwOrderdetail] as od
Left Join PartSummary ps 
    on od.Company = ps.company and od.OrderNumLine = ps.OrderNumLine
LEft Join LaborSummary ls 
    on od.Company = ls.company and od.OrderNumLine = ls.OrderNumLine
Where od.ProjectID = @ProjectID
      and od.Company = @Company
Group by od.Company,
         od.ProjectID,
 --        od.OrderNumLine

Warning: Null value is eliminated by an aggregate or other SET operation.

(15 rows affected)

Total execution time: 00:00:01.598

Company,ProjectID,OrderNumline,TotalPartCosts,TotalLaborCosts,TotalCosts
SOLAR,22221929,4000496-2,NULL,NULL,0.000
SOLAR,22221929,4000237-5,276901.060,NULL,276901.060
SOLAR,22221929,4000237-1,NULL,NULL,0.000
SOLAR,22221929,4000496-5,1529.850,NULL,1529.850
SOLAR,22221929,4000237-4,59269.470,38697.40980000000,97966.880
SOLAR,22221929,4000464-1,NULL,NULL,0.000
SOLAR,22221929,4000464-5,890.270,NULL,890.270
SOLAR,22221929,4000464-3,NULL,NULL,0.000
SOLAR,22221929,4000237-2,NULL,NULL,0.000
SOLAR,22221929,4000464-4,NULL,NULL,0.000


**Estimated** **Cost Example Query** for High River Project

In [21]:
Declare @ProjectID Varchar(50)
Set @ProjectID = '22221929';
Declare @Company Varchar(50)
Set @Company = 'Solar';

Select 
Company, 
ProjectID,
OrderNum,
OrderNumLine,
Sum(EstUnitCost*RequiredQty) as EstimatedJobDemands
FROM [ERPDB].[etl].[vwJobMtlChange]
Where ProjectID = @ProjectID
and Company = @Company
Group by Company, ProjectID, OrderNum, OrderNumline

Select 
Company, 
ProjectID,
Sum(EstUnitCost*RequiredQty) as EstimatedJobDemands
FROM [ERPDB].[etl].[vwJobMtlChange]
Where ProjectID = @ProjectID
and Company = @Company
Group by Company, ProjectID

(4 rows affected)

(1 row affected)

Total execution time: 00:00:20.843

Company,ProjectID,OrderNum,OrderNumLine,EstimatedJobDemands
SOLAR,2128153,4000149,4000149-1,1750.00000000000
SOLAR,2128153,4000149,4000149-3,0.00000000000
SOLAR,2128153,4000149,4000149-4,7150.00000000000
SOLAR,2128153,4000149,4000149-5,7483711.18972000000


Company,ProjectID,EstimatedJobDemands
SOLAR,2128153,7492611.18972000000


**Open Demand Example Query** for High River

In [5]:
Declare @ProjectID Varchar(50)
-- Set @ProjectID = '2128153';
Set @ProjectID = '22221929';
Declare @Company Varchar(50)
Set @Company = 'Solar';


With OpenJobDemands AS
(
Select
    Company, 
    ProjectID,
    JobNum,
    PartNum,
    [Description],
    OrderNum,
    OrderNumLine,
    (RequiredQty) as RequiredQuanity,
    (IssuedQty) as IssuedQuanty,
    EstUnitCost,
    (Case
         When (EstUnitCost*(RequiredQty-IssuedQty)) >= 0 Then  (EstUnitCost*(RequiredQty-IssuedQty))
         Else 0
    ENd) as OpenJobDemands,
    -- (EstUnitCost*(RequiredQty-IssuedQty)) as OpenJobDemands,
    (EstUnitCost*RequiredQty) as EstimatedJobDemands
FROM [ERPDB].[etl].[vwJobMtlChange]
Where ProjectID = @ProjectID
and Company = @Company
)

Select 
Company,
ProjectID,
OrderNumLine,
Sum(OpenJobDemands) as OpenJobDemands,
Sum(EstimatedJobDemands) as EstimatedJobDemands
From OpenJobDemands
Group by Company, ProjectID, OrderNumLine

(5 rows affected)

Total execution time: 00:00:07.802

Company,ProjectID,OrderNumLine,OpenJobDemands,EstimatedJobDemands
SOLAR,22221929,4000237-3,0.0000000000,0.00000000000
SOLAR,22221929,4000237-4,0.0000000000,0.00000000000
SOLAR,22221929,4000237-5,0.0000000000,61151.00112000000
SOLAR,22221929,4000464-5,0.0000000000,1113.77016000000
SOLAR,22221929,4000496-5,786.9240000000,786.92400000000


**Actual + Open Job Demand Query**

\-

In [6]:
Declare @ProjectID Varchar(50)
Set @ProjectID = '22221929';
Declare @Company Varchar(50)
Set @Company = 'Solar';

--Get Open Job Demands for Project for each job Assembly and Material sequence for each Order Line
With OpenJobDemandsDetail AS
(Select Company, 
    ProjectID,
    OrderNum,
    OrderNumLine,
    (Case
         When (EstUnitCost*(RequiredQty-IssuedQty)) >= 0 Then  (EstUnitCost*(RequiredQty-IssuedQty))
         Else 0
    ENd) as OpenJobDemands,
    (EstUnitCost*RequiredQty) as EstimatedJobDemands
FROM [ERPDB].[etl].[vwJobMtlChange]
),

--Summarize the Open Job Demands by orderline for comparison vs the actual costs on each order line
OpenJobDemands AS
(Select Company,
    ProjectID,
    OrderNum,
    OrderNumline,
    Sum(OpenJobDemands) as OpenJobDemands,
    Sum(EstimatedJobDemands) as EstimatedJobDemands
From OpenJobDemandsDetail
Group by Company, ProjectID, OrderNum, OrderNumline
),

-- Summarize the Part Costs on each Order Line
PartSummary
as (Select Company,
       ProjectID,
       OrderNum,
       Ordernumline,
       Sum(ExtCost) as OrderLinePartCosts
FROM [ERPDB].[etl].[vwPartTran]
Group by Company, ProjectID, OrderNum, OrderNumLine
),

-- Summarize the Labor Costs on each Order Line
LaborSummary
as (Select Company,
       ProjectID,
       OrderNum,
       Ordernumline,
       Sum((LaborHrs*LaborRate)+(BurdenHrs*BurdenRate)) as OrderLineLaborCosts
FROM [ERPDB].[etl].[vwLaborDtl]
Group by Company, ProjectID, OrderNum, OrderNumLine
),

--Summarize the Actual Costs on each Order line
ActualCost 
as (Select od.Company,
    od.ProjectID,
    od.OrderNum,
    od.Ordernumline,
    COALESCE(Sum(ps.OrderLinePartCosts),0) as TotalPartCosts,
    COALESCE(Sum(ls.OrderLineLaborCosts),0) as TotalLaborCosts,
    COALESCE(Sum(ps.OrderLinePartCosts),0)+COALESCE(Sum(ls.OrderLineLaborCosts),0) as ActualCosts
From [ERPDB].[etl].[vwOrderdetail] as od
Left Join PartSummary ps 
    on od.Company = ps.company and od.OrderNumLine = ps.OrderNumLine
Left Join LaborSummary ls 
    on od.Company = ls.company and od.OrderNumLine = ls.OrderNumLine
Group by od.Company, od.ProjectID, od.OrderNum, od.Ordernumline
)

Select 
    od.Company,
    od.ProjectID,
    Sum(OpenJobDemands) as OpenJobDemands,
    Sum(ActualCosts) as ActualCosts,
    Sum(COALESCE(OpenJobDemands,0) + COALESCE(ActualCosts,0)) as ActualCostsandOpenJobDemands,
    Sum(COALESCE(EstimatedJobDemands,0)) as EstimatedJobDemands
From [ERPDB].[etl].[vwOrderdetail] as od
Left Join ActualCost ac
on od.OrderNumLine = ac.OrderNumLine and od.Company = ac.Company
Left Join OpenJobDemands ojd
on od.OrderNumLine = ojd.OrderNumLine and od.Company = ojd.Company
Where od.ProjectID = @ProjectID
      and od.Company = @Company
Group by od.Company, od.ProjectID

Warning: Null value is eliminated by an aggregate or other SET operation.

(1 row affected)

Total execution time: 00:00:09.739

Company,ProjectID,OpenJobDemands,ActualCosts,ActualCostsandOpenJobDemands,EstimatedJobDemands
SOLAR,22221929,0.0000000000,386526.511,386526.511,63051.69528000000


**ProjPhase Order <span style="color: #0000ff;">Line</span> Summary**

In [2]:
Declare @ProjectID Varchar(50)
Set @ProjectID = '22221929';
Declare @Company Varchar(50)
Set @Company = 'Solar';

--Get Open Job Demands for Project for each job Assembly and Material sequence for each Order Line
With OpenJobDemandsDetail AS
(Select 
    Company, 
    ProjectID,
    OrderNum,
    OrderLine,
    (Case
         When (EstUnitCost*(RequiredQty-IssuedQty)) >= 0 Then  (EstUnitCost*(RequiredQty-IssuedQty))
         Else 0
    ENd) as OpenJobDemands,
    (EstUnitCost*RequiredQty) as EstimatedJobDemands
FROM [ERPDB].[etl].[vwJobMtlChange]
WHERE 1=1
    and ProjectID = @ProjectID
    and Company = @Company
)

Select 
    Company,
    ProjectID,
    OrderNum,
    OrderLine,
    SUM(OpenJobDemands) as OpenJobDemands,
    SUM(EstimatedJobDemands) as EstimatedJobDemands
From OpenJobDemandsDetail
WHERE 1=1
    and ProjectID = @ProjectID
    and Company = @Company
Group by Company, ProjectID, OrderNum, OrderLine

(5 rows affected)

Total execution time: 00:00:10.447

Company,ProjectID,OrderNum,OrderLine,OpenJobDemands,EstimatedJobDemands
SOLAR,22221929,4000237,3,0.0000000000,0.00000000000
SOLAR,22221929,4000237,4,0.0000000000,0.00000000000
SOLAR,22221929,4000237,5,766.2980000000,62321.00112000000
SOLAR,22221929,4000464,5,0.0000000000,1113.77016000000
SOLAR,22221929,4000496,5,0.0000000000,786.92400000000


In [4]:
Declare @ProjectID Varchar(50)
Set @ProjectID = '22221929';
Declare @Company Varchar(50)
Set @Company = 'Solar';

SELECT    
        Company,
        ProjectID,
        PhaseID,
        [Description],
        OrderNum,
        OrderLine,
        Level,
        SUM(BudTotLbrCost + BudTotBurCost + BudTotSubCost + BudTotMtlCost + BudTotMtlBurCost + BudTotODCCost) as BudgetedCosts,
        SUM(TotQuotODCCost + TotQuotLbrCost + TotQuotBurCost + TotQuotSubContCost + TotQuotMtlCost+ TotQuotMtlBurCost) as QuotedCosts,
        SUM([TotActLbrCost]+[TotActBurCost]+[TotActSubContCost]+[TotActMtlCost]+[TotActMtlBurCost]+[TotActODCCost]) as ActualCost,
        SUM(TotWBSPhaseRev) as SellPrice
FROM ERPDB.ERP.ProjPhase
    -- Where clause for Level only includes project phases at the top level which are already Summarized from lower levels
WHERE 1=1
    and ProjectID = @ProjectID
    and Company = @Company
GROUP BY Company, ProjectID, PhaseID, [Description], OrderNum, OrderLine, level

(20 rows affected)

Total execution time: 00:00:00.079

Company,ProjectID,PhaseID,Description,OrderNum,OrderLine,Level,BudgetedCosts,QuotedCosts,ActualCost,SellPrice
SOLAR,22221929,,Root Phase,0,0,1,0.000,0.000,0.000,0.000
SOLAR,22221929,1,Design,4000237,1,1,400.000,400.000,0.000,1250.000
SOLAR,22221929,1.1,C/O Design 1,0,0,2,0.000,0.000,0.000,0.000
SOLAR,22221929,2,Material,4000237,2,1,318959.050,378708.450,279321.180,530213.400
SOLAR,22221929,2.1,Const Phase 1,4000237,5,2,0.000,0.000,32602.600,0.000
SOLAR,22221929,2.10,C/O 3 Rev Ph 10,0,0,2,0.000,0.000,0.000,0.000
SOLAR,22221929,2.11,C/O 4 Rev Ph 11,0,0,2,0.000,0.000,0.000,0.000
SOLAR,22221929,2.2,Const Phase 2,4000237,5,2,0.000,0.000,60014.890,0.000
SOLAR,22221929,2.3,Const Phase 3,4000237,5,2,0.000,0.000,184283.570,0.000
SOLAR,22221929,2.4,C/O 1 Cons Ph4,4000464,5,2,0.000,0.000,890.270,0.000


Proving out that OrderNum/OrderLine are distinct to a project Phase.  Counts of the ProjectOrderLine CTE match across Count(\*) and Distinct Count (OrderNumLine)

In [11]:

Declare @Company Varchar(50)
Set @Company = 'Solar';

WIth ProjectORderLine AS
(SELECT DISTINCT
        ProjectID,
        PhaseID,
        OrderNum,
        OrderLine,
        CONCAT(Ordernum,OrderLine) as OrderNumLine,
        Level
FROM ERPDB.ERP.ProjPhase
    -- Where clause for Level only includes project phases at the top level which are already Summarized from lower levels
WHERE 1=1
    and Company = @Company
    and OrderNum <> 0
GROUP BY Company, ProjectID, PhaseID, [Description], OrderNum, OrderLine, level
)

Select Count(*)
FROM ProjectOrderLine;

-- Select Distinct Count (OrderNumline)
-- FROM ProjectOrderLine


(1 row affected)

Total execution time: 00:00:00.073

(No column name)
2851


In [34]:
Declare @ProjectID Varchar(50)
Set @ProjectID = '22221929';
Declare @Company Varchar(50)
Set @Company = 'Solar';


With OpenJobDemandsDetail AS
(Select 
    Company, 
    ProjectID,
    OrderNum,
    OrderLine,
    (Case
         When (EstUnitCost*(RequiredQty-IssuedQty)) >= 0 Then  (EstUnitCost*(RequiredQty-IssuedQty))
         Else 0
    ENd) as OpenJobDemands,
    (EstUnitCost*RequiredQty) as EstimatedJobDemands
FROM [ERPDB].[etl].[vwJobMtlChange]
WHERE 1=1
/*******************************************************************
Had to put these In here because of an error getting returned on 
converting Varchar to INT in this part of the query.  How to Solve?
*******************************************************************/
    and ProjectID = @ProjectID
    and Company = @Company
),

--Summarize the Open Job Demands by orderline for comparison vs the actual costs on each order line
OpenJobDemands AS
(Select 
    Company,
    ProjectID,
    OrderNum,
    OrderLine,
    SUM(OpenJobDemands) as OpenJobDemands,
    SUM(EstimatedJobDemands) as EstimatedJobDemands
From OpenJobDemandsDetail
Group by Company, ProjectID, OrderNum, OrderLine
)

SELECT DISTINCT
    pp.Company,
    pp.ProjectID,
    Case
        When CHARINDEX('.',pp.PhaseID) = 0 then LEN(pp.PhaseID)
        ELSE LEFT(pp.PhaseID,CHARINDEX('.',pp.PhaseID) - 1) 
    END as ParentPhaseID,
    pp.OrderNum,
    pp.OrderLine,
    SUM(ojd.OpenJobDemands) as OpenJobDemands,
    SUM(ojd.EstimatedJobDemands) as EstimatedJobDemands
FROM ERPDB.ERP.ProjPhase pp
LEFT JOIN OpenJobDemands ojd
    On pp.Company = ojd.Company and pp.OrderNum = ojd.OrderNum and pp.OrderLine = ojd.OrderLine
WHERE 1=1
    and pp.ProjectID = @ProjectID
    and pp.Company = @Company
    and pp.OrderNum <> 0
GROUP BY pp.Company, pp.ProjectID, pp.PhaseID, pp.OrderNum, pp.OrderLine

Warning: Null value is eliminated by an aggregate or other SET operation.

(10 rows affected)

Total execution time: 00:00:08.146

Company,ProjectID,ParentPhaseID,OrderNum,OrderLine,OpenJobDemands,EstimatedJobDemands
SOLAR,22221929,1,4000237,1,NULL,NULL
SOLAR,22221929,1,4000237,2,NULL,NULL
SOLAR,22221929,1,4000237,3,0.0000000000,0.00000000000
SOLAR,22221929,1,4000237,4,0.0000000000,0.00000000000
SOLAR,22221929,2,4000237,5,766.2980000000,62321.00112000000
SOLAR,22221929,2,4000464,2,NULL,NULL
SOLAR,22221929,2,4000464,5,0.0000000000,1113.77016000000
SOLAR,22221929,2,4000496,2,NULL,NULL
SOLAR,22221929,2,4000496,5,0.0000000000,786.92400000000
SOLAR,22221929,3,4000496,3,NULL,NULL


In [18]:
Select Top (1) *
From erp.projphase
Where Level = 3

Select Distinct Level
FRom erp.projphase

(1 row affected)

(4 rows affected)

Total execution time: 00:00:00.137

Company,ProjectID,PhaseID,Description,StartDate,DueDate,PercentComplete,DateComplete,PhaseStatus,Duration,WBSJobNum,ParentPhase,MSPTaskID,MSPPredecessor,UserMapData,BudTotLbrHours,BudTotBurHrs,BudTotLbrCost,BudTotBurCost,BudTotSubCost,BudTotMtlCost,BudTotMtlBurCost,TotEstLbrHrs,TotEstBurdenHrs,TotEstLbrCost,TotEstSubContCost,TotEstMtlCost,TotActLbrHrs,TotActBurHrs,TotActLbrCost,TotActBurCost,TotActSubContCost,TotActMtlCost,TotActMtlBurCost,ManEstCtcLbrHrs,ManEstCtcBurHrs,ManEstCtcLbrCost,ManEstCtcBurCost,ManEstCtcSubConCost,ManEstCtcMtlCost,ManEstCtcMtlBurCost,TotCtcLbrHours,TotCtcBurHours,TotCtcLbrCost,TotCtcBurCost,TotCtcSubConCost,TotCtcMtlCost,TotCtcMtlBurCost,TotQuotLbrHrs,TotQuotBurHrs,TotQuotLbrCost,TotQuotBurCost,TotQuotSubContCost,TotQuotMtlCost,TotQuotMtlBurCost,Level,DurationType,TotEstBurCost,TotEstMtlBurCost,RollChildMan,RollChildBud,SortSeq,MeasuredWorkID,TotQuotODCCost,TotEstODCCost,TotActODCCost,ManEstCTCODCCost,TotCTCODCCost,BudTotODCCost,TimeApprovalsMethod,TimeWFGroupID,ExpenseApprovalsMethod,ExpenseWFGroupID,PartNum,RevisionNum,SysRevID,SysRowID,InvMethod,RevMethod,OrderNum,OrderLine,WasRecInvoiced,LastBuildWBSPhaseAnalysisDate,PercentageOfCompletion,ToBeRecognizedLbrCost,ToBeRecognizedBurCost,ToBeRecognizedMtlCost,ToBeRecognizedSubCost,ToBeRecognizedMtlBurCost,ToBeRecognizedODCCost,ToBeRecognizedRevenue,RecognizeRevenueAtChildPhaseLevel,RollBudgetsToWBSPhase,TotWBSPhaseRev,SalesCatID,ActMtlNonJobCost,AsOfDate,BdnRecSeqPosted,BdnRecSeqLastAdded,BdnRevenueAutoToday,BillingToDate,BuildAnalysis,BurdenCostOfSales,BurdenLbrCstToDate,BurdenRecAutoCstTodate,BurdenRecManCstTodate,BurManPosted,BurPur,EstBurdenCost,EstBurdenHours,EstLaborCost,EstLaborHours,EstMtlBurdenCost,EstMtlCost,EstODCCost,EstSubcontractCost,LaborCostOfSales,LaborLbrCstToDate,LaborRecAutoCstTodate,LaborRecManCstTodate,LbrManPosted,LbrPur,MaterialCostOfSales,MaterialRecAutoCstTodate,MaterialRecManCstTodate,MtlBurdenCostOfSales,MtlBurdenRecAutoCstTodate,MtlBurdenRecManCstTodate,MtlBurManPosted,MtlBurPur,MtlManPosted,MtlPur,NextTmpInvcNum,ODCManPosted,ODCPur,ODCRecAutoCstToDate,ODCRecManCstTodate,RecManPosted,RecogToDtBilling,RecogToDtLbk,RecogToDtManual,RetentionDt,RevenueRecAutoToDate,RevenueRecManToDate,Reverse,RollManEstToCpte,SubCManPosted,SubConCostOfSales,SubConRecAutoCstTodate,SubConRecManCstTodate,SubPur,ConTotValue,DocConTotValue,Rpt1ConTotValue,Rpt2ConTotValue,Rpt3ConTotValue,ConTotValueNet,DocConTotValueNet,Rpt1ConTotValueNet,Rpt2ConTotValueNet,Rpt3ConTotValueNet
SOLAR,2024988,2.13,CONS PHS 1,NULL,2022-09-16,0.00,NULL,N,0.00,,2.1,,,,0.00,0.00,0.000,0.000,0.000,0.000,0.000,0.00,0.00,0.000,0.000,0.000,0.00,0.00,0.000,0.000,0.000,0.000,0.000,0.00,0.00,0.000,0.000,0.000,0.000,0.000,0.00,0.00,0.000,0.000,0.000,0.000,0.000,0.00,0.00,0.000,0.000,0.000,0.000,0.000,3,H,0.000,0.000,0,0,200,,0.000,0.000,0.000,0.000,0.000,0.000,,,,,,,0x00000003F39FEBB4,fa70b470-0389-4058-b224-a68ab8ce1bf8,CS,MAN,0,0,0,2023-03-24,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0,0,0.000,,0.000,2023-03-24,0,0,0.000,0.000,0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,0.000,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,,0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


Level
0
3
1
2


**Project info** for example project High River

In [1]:
Declare @ProjectID Varchar(50)
Set @ProjectID = '2128153';
Declare @Company Varchar(50)
Set @Company = 'Solar';

Select p.Company,
       p.ProjectID,
       p.[Description] as ProjectName,
       p.ActiveProject,
       p.SalesCatID,
       sc.[Description] as SalesCategory,
       p.ConProjMgr,
       eb.Name as PMName,
       p.ConBTCustNum,
       c.Name as CustomerName
FROM [ERPDB].[Erp].[Project] p
    LEFT JOIN [ERPDB].[Erp].[SalesCat] sc
        on p.company = sc.company
           and p.SalesCatID = sc.SalesCatID
     LEFT JOIN [ERPDB].[Erp].[EmpBasic] eb
        on p.ConProjMgr = eb.EmpID
          and p.company = eb.company
    LEFT JOIN [ERPDB].[Erp].[Customer] c
        on p.ConBTCustNum = c.CustNum
          and p.Company = c.Company
where p.Company = @Company
      and ProjectID = @ProjectID

(1 row affected)

Total execution time: 00:00:00.160

Company,ProjectID,ProjectName,ActiveProject,SalesCatID,SalesCategory,ConProjMgr,PMName,ConBTCustNum,CustomerName
SOLAR,2128153,High River,1,AGIL,Agile,27302,Richard W Van Fleet,474,NEXTERA ENERGY RESOURCES LLC


**Ship State Query** 

Note: 
- this looks like it is coming from the OrderHead and not the Project.  In EPICOR The Project ID is assigned to the Order Line and a Project can have Many orders.  So in order to get a single Ship to in view there would have to selection on only one through likely a window function of some sort
- With additional searhing there is a Orderhed table in the dbo schema which has a custom field to attach the Project is.  Therefore, you can get orderhed.ProjectID_c form the dbo schema table and use that to join to the Ship to without having to use the OrderDetail table.  The many orders per project issue still arises though

In [6]:
Declare @ProjectID Varchar(50)
Set @ProjectID = '22221929';
Declare @Company Varchar(50)
Set @Company = 'Solar';

Select Distinct
    od.Company,
    od.ProjectID,
    od.OrderNum,
    oh.shiptoNum,
    st.State as ShipToState,
    st.Zip as ShipToZip
  FROM [ERPDB].[etl].vwOrderDetail od
Left JOIN [ERPDB].[Erp].[orderhed] oh
    on od.ordernum = oh.ordernum and od.company=oh.Company
Left Join [ERPDB].[Erp].[ShipTo] st 
    on oh.shiptoNum = st.ShipToNum and oh.company = st.company and oh.CustNum = st.CustNum

  WHERE od.Company = @Company
   and ProjectID = @ProjectID

   Select Distinct
    oh.Company,
    oh.ProjectID_c as ProjectID,
    oh.OrderNum,
    oh.shiptoNum,
    st.State as ShipToState,
    st.Zip as ShipToZip
  FROM [ERPDB].[dbo].[orderhed] oh
Left Join [ERPDB].[Erp].[ShipTo] st 
    on oh.shiptoNum = st.ShipToNum and oh.company = st.company and oh.CustNum = st.CustNum

  WHERE oh.Company = @Company
   and oh.ProjectID_c = @ProjectID

(3 rows affected)

(3 rows affected)

Total execution time: 00:00:00.211

Company,ProjectID,OrderNum,shiptoNum,ShipToState,ShipToZip
SOLAR,22221929,4000237,04654,ME,04654
SOLAR,22221929,4000464,04654,ME,04654
SOLAR,22221929,4000496,04654,ME,04654


Company,ProjectID,OrderNum,shiptoNum,ShipToState,ShipToZip
SOLAR,22221929,4000464,04654,ME,04654
SOLAR,22221929,4000237,04654,ME,04654
SOLAR,22221929,4000496,04654,ME,04654


**Billing Query example** for High River

In [15]:
Declare @ProjectID Varchar(50)
Set @ProjectID = '22221929';
Declare @Company Varchar(50)
Set @Company = 'Solar';
 
 With InvoiceSummary
 as (Select Company,
    OrderNum,
   Sum(InvoiceAmt) as Billed,
   Sum(InvoiceBal) as InvoiceBalance,
   (Sum(InvoiceAmt) - Sum(INvoiceBal)) as CustomerPaid
    FROM [ERPDB].[Erp].[InvcHead]

  Group by Company, OrderNum
 )

Select
oh.Company,
oh.ProjectID_c as ProjectID,
oh.OrderNum,
oh.Division_c,
Sum(inv.Billed) as Billed,
Sum(inv.InvoiceBalance) as InvoiceBalance,
sum(inv.CustomerPaid) as CustomerPaid
FROM [ERPDB].[dbo].[orderhed] oh
Left Join InvoiceSummary inv
on oh.OrderNum = inv.OrderNum and oh.Company = inv.Company
Where oh.Company = @Company
  and oh.ProjectID_c = @ProjectID
Group by oh.Company, oh.ProjectID_c, oh.OrderNum, oh.Division_c

(3 rows affected)

Total execution time: 00:00:00.122

Company,ProjectID,OrderNum,Division_c,Billed,InvoiceBalance,CustomerPaid
SOLAR,22221929,4000464,0,1344.000,1344.000,0.000
SOLAR,22221929,4000237,0,715418.000,595554.400,119863.600
SOLAR,22221929,4000496,0,NULL,NULL,NULL


**Sales Categories**

- Sales Categories are put on the project for the new Screw Projects (Agile, TerraTrak, Tracker Interface)
- legacy rbi Wave/Canopy/Sunflower put the Salescategory on the OrderDtl and the Division is on the dbo.Orderhed.
- Agile/TerraTrak/TrackerInterface also get the SalesCategory on the OrderDtl but the Division is not on the dbo.Orderhed or the Project
- for screw Projects Epicor automatically posts based on the Product Group (is this = to SalesCategory?) on the SO line (Cost) and the GL Control on the WBS Phase (Revenue)
- Some Sales Categories will be on mulitple divisions (example = GM).  When this is the case the division should reflect what is on the Orderhead

In [9]:
--List of Sales Categories used on Project table. 
Select distinct 
       sc.Company
    ,sc.[Description] as SalesCategory
FROM [ERPDB].[Erp].[Project] p
    LEFT JOIN [ERPDB].[Erp].[SalesCat] sc
        on p.company = sc.company
           and p.SalesCatID = sc.SalesCatID
Where sc.Company = 'Solar'
Order by sc.Company


-- List of distinct Sales Categories for Solar
Select distinct 
       sc.Company
    ,sc.[Description] as SalesCategory
FROM [ERPDB].[Erp].[SalesCat] sc
Where sc.Company = 'Solar'
Order by sc.Company

-- list of distinct sales categories for Solar orders which looks like it puts the sales category on the new Agile/Terratrak/TrackerInterface product types but doesn't put the division on the order head
Select Distinct 
    od.Company,
    oh.Division_c,
    od.SalesCatID,
    sc.[Description] as SalesCategory
   FROM [ERPDB].[etl].vwOrderDetail od
Left JOIN [ERPDB].[dbo].[orderhed] oh
    on od.ordernum = oh.ordernum and od.company=oh.Company
LEFT JOIN [ERPDB].[Erp].[SalesCat] sc
     on od.company = sc.company
           and od.SalesCatID = sc.SalesCatID
Where sc.Company != 'APEKS'
Order by od.Company, Division_c


(3 rows affected)

(84 rows affected)

(58 rows affected)

Total execution time: 00:00:00.391

Company,SalesCategory
SOLAR,Agile
SOLAR,TerraTrak
SOLAR,TrkrInterface


Company,SalesCategory
SOLAR,Agile
SOLAR,BA CPST CTC D
SOLAR,BA CPST CTC L
SOLAR,BA CPST CTC M
SOLAR,BA CPST CTC M&L
SOLAR,BA IB 4SQTC D
SOLAR,BA IB 4SQTC L
SOLAR,BA IB 4SQTC M
SOLAR,BA IB 4SQTC M&L
SOLAR,BA IB CTC D


Company,Division_c,SalesCatID,SalesCategory
SOLAR,3,GI4I,GR IB 4SQTC M&L
SOLAR,3,BICM,BA IB CTC M
SOLAR,3,GI4M,GR IB 4SQTC M
SOLAR,3,GICI,GR IB CTC M&L
SOLAR,3,GM,Ground Mount
SOLAR,3,PSTI,PKG SUR IB M&L
SOLAR,3,RM,Roof Mount
SOLAR,3,GI4D,GR IB 4SQTC D
SOLAR,3,GCCI,GR CPST CTC M&L
SOLAR,3,GMRD,GR MNT R&D


**Sales Category to Division Table Logic**

In [ ]:
-- Logic to Map SalesCatID to Division with Case Statment for Agile, TerraTrak, TrackerInterface Divisions included
Select Distinct 
    od.Company,
    CASE
        WHEN od.SalesCatID = 'TT' THEN 8
        WHEN od.SalesCatID = 'TINT' THEN 4
        WHEN od.SalesCatID = 'AGIL' THEN 2
        ELSE oh.Division_c
    END as Division,
    od.SalesCatID,
    sc.[Description] as SalesCategory
   FROM [ERPDB].[etl].vwOrderDetail od
Left JOIN [ERPDB].[dbo].[orderhed] oh
    on od.ordernum = oh.ordernum and od.company=oh.Company
LEFT JOIN [ERPDB].[Erp].[SalesCat] sc
     on od.company = sc.company
           and od.SalesCatID = sc.SalesCatID
Where sc.Company != 'APEKS'
Order by od.Company, Division

Example of How SalesCategory ID lands on ProjectPhase or WBS Phase of a Project

In [17]:
Declare @ProjectID Varchar(50)
Set @ProjectID = '22221929';
Declare @Company Varchar(50)
Set @Company = 'Solar';

SELECT 
       Company,
       ProjectID,
       PhaseID
       OrderNum,
       OrderLine,
       Wbsjobnum,
       SalesCatID,
       [Level],
       (BudTotLbrCost) as BudgetedLaborCosts,
       (BudTotBurCost) as BudgetedBurdenCosts,
       (BudTotSubCost) as BudgetedSubCosts,
       (BudTotMtlCost) as BudgetedMaterialCosts,
       (BudTotMtlBurCost) as BudgetedMaterialBurdenCosts,
       (BudTotODCCost) as BudgetedOtherDirectCosts,
       (BudTotLbrCost + BudTotBurCost + BudTotSubCost + BudTotMtlCost + BudTotMtlBurCost + BudTotODCCost) as EstimatedCosts
From ERPDB.ERP.ProjPhase
Where ProjectID = @ProjectID
      and company = @Company
      and [level] = 1

(5 rows affected)

Total execution time: 00:00:00.046

Company,ProjectID,OrderNum,OrderLine,Wbsjobnum,SalesCatID,Level,BudgetedLaborCosts,BudgetedBurdenCosts,BudgetedSubCosts,BudgetedMaterialCosts,BudgetedMaterialBurdenCosts,BudgetedOtherDirectCosts,EstimatedCosts
SOLAR,22221929,,0,,,1,0.000,0.000,0.000,0.000,0.000,0.000,0.000
SOLAR,22221929,1,1,,,1,0.000,0.000,0.000,0.000,0.000,400.000,400.000
SOLAR,22221929,2,2,,AGIL,1,0.000,0.000,0.000,0.000,-59749.400,378708.450,318959.050
SOLAR,22221929,3,3,,AGIL,1,0.000,0.000,0.000,0.000,0.000,57345.820,57345.820
SOLAR,22221929,4,4,,AGIL,1,0.000,0.000,0.000,0.000,0.000,78762.030,78762.030


List of ERP Tables used to generate data

In [ ]:
-- OrderDetail view was created to concat teh Change Date and Change Time to get a watermark and create OrderNumLine Concat
[ERPDB].[etl].[vwOrderdetail]
    [ERPDB].[dbo].[OrderDtl]
--Part Tran view has the OrderNum, OrderNHumLine and Project ID joined to it and filter our 'MFG-VAR' TranType and get the ChangeDAteTime watermark
[ERPDB].[etl].[vwPartTran]
    [ERPDB].[Erp].[PartTran]
    [ERPDB].[etl].[vwOrderDetail]
-- vwLaborDtl is joined on OrderNumLine
[ERPDB].[etl].[vwLaborDtl]    
    [ERPDB].[Erp].[LaborDtl]
    [etl].[vwOrderDetail]
--JobMtlChange view has the indented are the tables joined in the view to get the last updated date watermark for ETL
[ERPDB].[etl].[vwJobMtlChange]
    [ERPDB].[Erp].[JobMtl] 
    [ERPDB].[Ice].[ChgLog] --used to the DateStamped on from the Change log
    [ERPDB].[Erp].[PartTran] -- used to last Transaction Date on the Job Assembly Sequence
    [ERPDB].[Erp].[JobHead] --Used to get the Job Created Date

[ERPDB].[ERP].[ProjPhase] -- used to get Quoted and Budgeted Costs
[ERPDB].[Erp].[Project] -- Used to get Project info to join to SalesCAt, EmpBasic, and Customer
[ERPDB].[Erp].[SalesCat] -- Used to get Sales Category
[ERPDB].[Erp].[EmpBasic] -- Used to get PM Info
[ERPDB].[Erp].[Customer] -- Used to get Customer Info
[ERPDB].[Erp].[orderhed] --Used to Join Ship to State
[ERPDB].[Erp].[ShipTo] -- Used to get Ship to State
[ERPDB].[Erp].[InvcHead] -- Used to get Billing Summary
